## SeqNeo
- Stage: Cambrian
- Version: Pomoria

#### Related Commands
- tensorboard --logdir runs
- nohup jupyter lab > jupyter.log &

In [1]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [2]:
import os
import itertools
import copy
import random
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [3]:
import tyrell.spec as S
from tyrell.decider import Example

# Morpheus Version
from MorpheusInterpreter import *
from ProgramSpace import *

In [4]:
torch.__version__

'1.0.0'

In [5]:
class SeqNeo(nn.Module):
    def __init__(self, p_config=None):
        super(SeqNeo, self).__init__()
        self.config = p_config
        
        self.fn_embedding = nn.Embedding(
            self.config["fn_vocab_size"],
            self.config["fn_dim"],
            padding_idx=0,
        )
        
        self.encoder1 = nn.Linear(
            self.config["encoder"]["input_dim"],
            2048,
        )
        self.encoder2 = nn.Linear(
            2048,
            self.config["encoder"]["output_dim"],
        )
        
        self.decoder = nn.GRUCell(
            self.config["decoder"]["input_dim"],
            self.config["decoder"]["hidden_dim"],
            bias=True,
        )
        self.classifier = nn.Linear(
            self.config["decoder"]["hidden_dim"],
            self.config["decoder"]["output_dim"],
        )
        
    def decode(self, pin_exp, pout_exp, p_hidden, pfn_context):
        tmp_exp = self.encode(pin_exp, pout_exp) # (B=1, encoder_output_dim)
        tmp_fn = self.fn_embedding(pfn_context) # (B=1, fn_dim)
        tmp_input = torch.cat([tmp_exp,tmp_fn],dim=1)
        
        if p_hidden is None:
            # very first
            tmp_hidden = self.decoder(tmp_input)
        else:
            # not the very first
            tmp_hidden = self.decoder(tmp_input, p_hidden)
        # (B=1, decoder_hidden_dim)
        
        tmp_output = torch.log_softmax(
            self.classifier(tmp_hidden),
            dim=1
        ) # (B=1, decoder_output_dim)
        
        # first hidden, then output
        return (tmp_hidden, tmp_output)
        
        
    def encode(self, pin_exp, pout_exp):
        # pin_exp/pout_exp: (B=1, abs_dim)
        tmp_exp = torch.cat([pin_exp,pout_exp],dim=1) # (B=1, encoder_input_dim=abs_dim*2)
        tmp_out1 = F.relu(self.encoder1(tmp_exp))
        tmp_out2 = F.relu(self.encoder2(tmp_out1))
        return tmp_out2 # (B=1, encoder_output_dim)
    

In [6]:
# replace certain node id with certain value
def modify_shell(p_shell, p_id_from, p_id_to):
    d_prod = p_shell[0]
    d_rhs = p_shell[1]
    ld_rhs = [p_id_to if d_rhs[i]==p_id_from else d_rhs[i]
             for i in range(len(d_rhs))]
    return (d_prod, tuple(ld_rhs))

def get_sketch_from_prog(p_prog):
    pp = str(p_prog)
    s = pp.index("@")
    pp = pp[:s]
    pl = pp.split("(")
    pl = [p for p in pl if p][::-1]
    return tuple(pl)

# using default path and solutions
from benchmarks.pldi17_sol import *
def load_benchmark(p_id, p_sourceps, p_builder):
    d_prog = p_builder._from_sexp(solutions[p_id])
    d_input = p_sourceps.interpreter.init_tbl("./benchmarks/pldi17/p{}_input1.csv".format(p_id))
    p_sourceps.interpreter.create_shadow(d_input)
    d_output = p_sourceps.interpreter.init_tbl("./benchmarks/pldi17/p{}_output1.csv".format(p_id))
    p_sourceps.interpreter.create_shadow(d_output)
    d_ps = ProgramSpace(
        p_sourceps.spec, p_sourceps.interpreter, [d_input], d_output,
    )
    d_ps.init_by_prog(d_prog)
    return d_ps

def SeqTrain(p_config, p_sourceps, p_model, p_dataset, p_optim):
    print("# Start SeqTrain...")
    p_builder = D.Builder(p_sourceps.spec)
    
#     DBG_SKETCH = {
#         # ("gather","unite","spread"):0,
#         ("neg_gather","unite","spread"):0,
#         ("neg_gather","group_by","summarise"):0,
#     }
    
    batch_list = []
    for d_epoch in range(p_config["train"]["n_epoch"]):
        epoch_loss_list = []
        for d_episode in range(p_config["train"]["n_episode"]):
            print("\r# EP:{}/{}, loss:{:.2f}".format(
                d_epoch, d_episode,
                sum(epoch_loss_list)/len(epoch_loss_list) if len(epoch_loss_list)>0 else -1,
            ),end="")
            p_model.train()
            
            if isinstance(p_dataset,list):
                # ====== option 1: sample from dataset ====== #
                while True:
                    eid = random.choice(range(len(p_dataset)))
                    data_prog, data_str_example = p_dataset[eid]
                    data_example = Example(
                        input=[p_sourceps.interpreter.load_data_into_var(p) for p in data_str_example.input],
                        output=p_sourceps.interpreter.load_data_into_var(data_str_example.output),
                    )
                    ps_solution = ProgramSpace(
                        p_sourceps.spec, p_sourceps.interpreter, data_example.input, data_example.output,
                    )
                    ps_solution.init_by_prog(data_prog)
                    break
            elif isinstance(p_dataset,int):
                # print("# load Morpheus benchmark #")
                # ====== option 2: load Morpheus benchmarks ====== #
                ps_solution = load_benchmark(p_dataset, p_sourceps, p_builder)
            else:
                # ====== option 2: sample from generator ====== #
                ps_solution = p_dataset.get_new_chain_program(
                    p_config["train"]["n_size"] + 1 # depth=size+1
                )

            # solution self-check
            if ps_solution.check_eq() is None:
                continue
                print("ERROR, SOLUTION NOT CONSISTENT!")

            selected_neurons = []

            # initialize a new Program Space
            ps_current = ProgramSpace(
                p_sourceps.spec, p_sourceps.interpreter, ps_solution.inputs, ps_solution.output,
            )
            # then initialize a shell template
            tmp_shell_list = ps_current.get_neighboring_shells()
            tmp_node_to_replace = ps_current.node_dict["ParamNode"][0] # for chain only
            # replace the Param Node id in shells with -1 to make them templates
            template_list = [
                modify_shell(tmp_shell_list[i],tmp_node_to_replace,-1)
                for i in range(len(tmp_shell_list))
            ]

            hidden_current = None # hidden state, initialized to None
            fn_context = 0
            for d_step in range(len(ps_solution.shells)):

                # ### assume chain execution, so only 1 possible returns
                # ### at d_step=0, this should be input[0]
                id_current = ps_current.get_strict_frontiers()[0]
                var_current = ps_current.node_list[id_current].ps_data # need the real var name in r env
                var_output = ps_current.output

                map_current = p_sourceps.interpreter.camb_get_pomoria(var_current)
                map_output = p_sourceps.interpreter.camb_get_pomoria(var_output)

                # make current shell list
                current_shell_list = [
                    modify_shell(template_list[i],-1,id_current)
                    for i in range(len(template_list))
                ]
                current_shell_list = [None] + current_shell_list # None for <SOS> in nn embedding

                # wrap in B=1
                if use_cuda:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float)).cuda()
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float)).cuda()
                    td_context = Variable(torch.tensor([fn_context],dtype=torch.long)).cuda()
                else:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float))
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float))
                    td_context = Variable(torch.tensor([fn_context],dtype=torch.long))

                # (B=1, fn_vocab_size)
                hidden_current, td_pred = p_model.decode(td_current, td_output, hidden_current, td_context)
                # supervised, assign selection directly
                tmp_id = current_shell_list.index(ps_solution.shells[d_step])
                fn_context = tmp_id

                # update ps_current
                update_status = ps_current.add_neighboring_shell(
                    current_shell_list[tmp_id]
                )
                
                if not update_status:
                    # something wrong
                    raise Exception("Can't even update PS under supervision?")
                
                selected_neurons.append(td_pred[0,tmp_id])

            # <END_FOR_STEP>
            if ps_current.check_eq() is None:
                # something wrong, should've been solved under supervision
                raise Exception("Can't even solve under supervision?")

            episode_loss_list = []
            for i in range(len(selected_neurons)):
                episode_loss_list.append(
                    (+1.0)*(-selected_neurons[i])
                )
            batch_list += episode_loss_list
            
            episode_loss = sum(episode_loss_list)
            epoch_loss_list.append(episode_loss)
            
            if d_episode%p_config["train"]["batch_size"]==0:
                p_optim.zero_grad()
                batch_loss = sum(batch_list)/len(batch_list)
                batch_loss.backward()
                p_optim.step()
                batch_list = []

                
        # <END_FOR_EPISODE>  
        print()
        if d_epoch % 5 == 0:
            torch.save(p_model.state_dict(),"./saved_models/0813CambAgent_Thomas_Size3_ep{}.pt".format(d_epoch))
    # <END_FOR_EPOCH>
    

In [7]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/camb6.tyrell')
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
)
# dumb variable to help infer the shells
m_ps = ProgramSpace(
    m_spec, m_interpreter, [None], None,
)

In [8]:
m_config = {
    "abs_dim": 15*9+3,
    "fn_dim": 128,
    "fn_vocab_size": len(m_ps.get_neighboring_shells())+1, # with one more <SOS>
    "encoder":{
        "input_dim": None,
        "output_dim": 1024, # hidden state
    },
    "decoder":{
        "input_dim": None,
        "hidden_dim": None,
        "output_dim": None,
    },
    "train":{
        "n_epoch":500, # #epoches in total
        "n_episode": 100, # #problems per epoch
        "n_size": 3,
        "batch_size": 16,
    },
    "test":{
        "beam_size": 40,
        "sprout_size": 40,
    },
}
m_config["encoder"]["input_dim"] = m_config["abs_dim"]*2
m_config["decoder"]["input_dim"] = \
    m_config["encoder"]["output_dim"] + \
    m_config["fn_dim"]
m_config["decoder"]["hidden_dim"] = m_config["encoder"]["output_dim"]
m_config["decoder"]["output_dim"] = m_config["fn_vocab_size"]

In [9]:
with open("./0812Size3RelaxedFiltered.pkl","rb") as f:
    m_dataset = pickle.load(f)

In [10]:
seq_neo = SeqNeo(p_config=m_config)
if use_cuda:
    seq_neo = seq_neo.cuda()
optimizer = torch.optim.Adam(list(seq_neo.parameters()))

In [11]:
# SeqTrain(m_config, m_ps, seq_neo, m_generator, optimizer)
SeqTrain(m_config, m_ps, seq_neo, m_dataset, optimizer)
# SeqTrain(m_config, m_ps, seq_neo, 53, optimizer)

# Start SeqTrain...
# EP:0/99, loss:16.09
# EP:1/99, loss:14.77
# EP:2/99, loss:13.46
# EP:3/99, loss:12.51
# EP:4/99, loss:11.92
# EP:5/99, loss:10.85
# EP:6/99, loss:10.42
# EP:7/99, loss:10.32
# EP:8/99, loss:10.09
# EP:9/99, loss:9.88
# EP:10/99, loss:9.77
# EP:11/99, loss:9.18
# EP:12/99, loss:8.99
# EP:13/99, loss:8.96
# EP:14/99, loss:9.20
# EP:15/99, loss:8.62
# EP:16/99, loss:8.69
# EP:17/99, loss:8.52
# EP:18/99, loss:8.28
# EP:19/99, loss:8.50
# EP:20/99, loss:7.96
# EP:21/99, loss:7.84
# EP:22/99, loss:8.09
# EP:23/99, loss:8.35
# EP:24/99, loss:8.27
# EP:25/99, loss:7.70
# EP:26/99, loss:7.49
# EP:27/99, loss:7.83
# EP:28/99, loss:7.55
# EP:29/99, loss:7.58
# EP:30/99, loss:7.30
# EP:31/99, loss:7.42
# EP:32/99, loss:7.26
# EP:33/99, loss:6.98
# EP:34/99, loss:7.13
# EP:35/99, loss:7.11
# EP:36/99, loss:7.52
# EP:37/99, loss:6.65
# EP:38/99, loss:7.02
# EP:39/99, loss:6.85
# EP:40/99, loss:7.09
# EP:41/99, loss:6.59
# EP:42/99, loss:6.97
# EP:43/99, loss:7.10
# EP:44/99, l

KeyboardInterrupt: 

In [ ]:
# SeqBeam(m_config, m_ps, seq_neo, m_dataset)